In [16]:
import pandas as pd

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# 1. 데이터셋 불러오기
df = pd.read_csv("synthetic_brain_hemorrhage_data.csv")
df.head()

# 2. 입력(독립, X, X_), 정답(종속, y)
## 지도학습은 문제와 정답이 반드시 존재
X = df.drop("label", axis=1)
y = df["label"]

# 3. 학습(훈련) 데이터 / 테스트(검증) 데이터
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. XGBoost 모델 정의 및 학습 => 결정트리를 이용해서
## 모델 정의
model = XGBClassifier(
    n_estimators = 300, # 트리개수
    learning_rate = 0.05,  #학습률
    max_depth = 4, #트리 깊이
    eval_metric = "logloss",
    random_state = 42
)
## 훈련
model.fit(X_train, y_train) # 머신러닝의 훈련 정답

# 5.평가 : 
# 훈련데이터(X_) -> 훈련 종료 후 -> X_test(검증 문제)를
# 훈련 모델에 넣어서 결과를 확인한 결과를 실제 검증 답(y_test)
# 와 비교를 하는 게 평가

y_pred = model.predict(X_test)

print( classification_report(y_test, y_pred) )



              precision    recall  f1-score   support

           0       0.63      0.36      0.46        33
           1       0.74      0.90      0.81        67

    accuracy                           0.72       100
   macro avg       0.69      0.63      0.64       100
weighted avg       0.70      0.72      0.70       100



# 하이퍼 파라미터 튜닝 - GridSearch

In [15]:
from sklearn.model_selection import GridSearchCV

# 1. 모델 정의
xgb = XGBClassifier(
    eval_metric = "logloss",
    random_state = 42
)

params = {
    "n_estimators" : [200, 300, 400],
    "learning_rate" : [0.01, 0.05, 0.1],
    "max_depth" : [3, 4, 5],
}

grid_result = GridSearchCV(
    estimator = xgb, #사용 모델,
    param_grid = params,
    cv = 3
)

grid_result.fit(X_train, y_train) #243 * 3

print("최적의 하이퍼 파라미터 : ", grid_result.best_params_)

최적의 하이퍼 파라미터 :  {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 400}
